In [1]:
import numpy as np
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from scrapping.scrapping_tools import *
from utils.dict_utils import *


{}
{}


In [2]:
orateurs_datas = {}
not_found = []

In [7]:
full_datas= []
for k in range(1,3):
    L = get_links_page(k)
    print("page = ", k )
    for c,i in enumerate(L):
        xml_l = get_xml(i)
        if(not xml_l):
            continue
        dt = extract_xml(xml_l, orateurs_datas, not_found)
        full_datas += dt
        #print("dt= ", dt)

page =  1
page =  2


In [8]:
D= merge_dict(orateurs_datas.values())
df = pd.DataFrame(D)
df

""


In [5]:
D= merge_dict(full_datas)
df = pd.DataFrame(D)
df

""


In [6]:
df_g = df.groupby("group").apply(lambda x: x.sum())["texte"].reset_index()

KeyError: 'group'

In [ ]:
osef_words = ["président", "article", "banc", "français","loi", "texte", "groupe", "applaudissement", "monsieur", "ministre", "amendement", "question", "état", "gouvernement"]
def filter_text(text):
    nlp = spacy.load("fr_core_news_sm")
    nlp.max_length = 1e25
    doc = nlp(text)
    conserved_words = []
    for token in doc:
        if(token.pos_ == "NOUN" and not token.is_stop and token.is_alpha):
            if(token.lemma_ not in osef_words and token.text not in osef_words):
                conserved_words.append(token.lemma_)
    return conserved_words

In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
    

In [ ]:
for group, texte in zip(df_g["group"], df_g["texte"]):
    corpus = filter_text(texte)
    freq = get_top_n_words(corpus, n = 25)
    print("========")
    print(f"group: {group} : ")
    for f in freq:
        print(f[0], " : ", f[1])
    print("========")


In [ ]:
def count_term(text, term):
    c = 0
    for k in text:
        if( k.lower() == term.lower()):
            c = c + 1
    return c


In [ ]:
displ = {
    "group":[], 
    "word":[],
    "number":[]
}
for group, texte in zip(df_g["group"], df_g["texte"]):
    print("group: ", group)
    f_text = filter_text(texte)
    print("Travail : ", count_term(f_text,"travail"))
    displ["word"].append("Travail")
    displ["group"].append(group)
    displ["number"].append(count_term(f_text,"travail"))
    print("Droit : ", count_term(f_text,"droit"))
    displ["word"].append("Droit")
    displ["group"].append(group)
    displ["number"].append(count_term(f_text,"droit"))


In [ ]:
df_g = pd.DataFrame(displ)

In [ ]:
df_g["group"] = df_g["group"].replace("Mouvement Démocrate et apparentés", "MODEM")
df_g["group"] = df_g["group"].replace("Mouvement Démocrate (MoDem) et Démocrates apparentés", "MODEM")
df_g["group"] = df_g["group"].replace("La France insoumise", "LFI")
df_g["group"] = df_g["group"].replace("Agir ensemble", "AE")
df_g["group"] = df_g["group"].replace("La République en Marche", "LREM")
df_g["group"] = df_g["group"].replace("Les Républicains", "LR")
df_g["group"] = df_g["group"].replace("Libertés et Territoires", "LT")
df_g["group"] = df_g["group"].replace("Non inscrit", "NI")
df_g["group"] = df_g["group"].replace("Nouvelle Gauche", "NG")
df_g["group"] = df_g["group"].replace("Socialistes et apparentés", "SOC")
df_g["group"] = df_g["group"].replace("UDI et Indépendants", "UDI")
df_g["group"] = df_g["group"].replace("UDI, Agir et Indépendants", "UDI")

df_g["group"] = df_g["group"].replace("Socialiste", "SOC")


df_g["group"] = df_g["group"].replace("Gauche démocrate et républicaine", "GDR")


In [ ]:
df_g = df_g.groupby(["group","word"]).sum().reset_index()
df_g

In [ ]:
s = df_g[df_g.group == "AE"]["number"].sum()
df_g[df_g.group == "AE"]["number"] = df_g[df_g.group == "AE"]["number"] / s

In [ ]:
df_g[df_g.group == "AE"]["number"]

In [ ]:

import seaborn as sns
sns.set_theme(style="whitegrid")

In [ ]:
g = sns.catplot(
    data=df_g, kind="bar",
    x="group", y="number", hue="word",
    ci="sd", palette="dark", alpha=.6, height=6
)
g.despine(left=True)
g.set_axis_labels("", "Number of occurrences")
g.legend.set_title("")

In [ ]:
fig = g.fig
fig.savefig('droit_travail.png', dpi=fig.dpi)